In [11]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
from logging import error, warn
from gsw.GSWriter import GSWriter
from constants import CONSTANTS


ImportError: cannot import name 'CONSTANTS' from 'constants' (/home/jinwei/wise/geoscenarioserver_new/dataset_api/constants.py)

# InteractionDatasetLoader
## Task breakdown
- load data from the dataset


In [10]:
dataset_config = {}
dataset_config["base_dir"] = BASE_DIR
dataset_config["version"] = DATASET_VERSION
dataset_config["recording_map"] = RECORDING_MAP

In [3]:
class InteractionDataset():
   

    def __init__(self, config, train=True):
        self.config = config
        self.train = train
        
        if self.train:
            self.base_dir = config["base_dir"]
            self.recording_map = config["recording_map"]
            self.csv_dir_list = [self.get_track_directory(recording) for recording in self.recording_map]
            self.osm_dir_list = [self.get_map_directory(recording) for recording in self.recording_map]
            
        else:
            self.base_dir = config["base_dir"]
            self.recording_map = config["recording_map"]
            self.csv_dir_list = [self.get_track_directory(recording) for recording in self.recording_map]
            self.osm_dir_list = [self.get_map_directory(recording) for recording in self.recording_map]
    
    def get_track_directory(self, recording):
        return self.base_dir+'train/'+recording+'_train.csv' if self.train else self.base_dir+'val/'+recording+'_val.csv'
        
    def get_map_directory(self, recording):
        return self.base_dir+'maps/'+recording+'.osm'
            
    def get_recording_df(self, recording):
        assert recording in self.recording_map
        return pd.read_csv(self.get_track_directory(recording))
    
    def get_scenario_df_of_recording(self, scenario_id, recording_df):
        scenario_id_list = recording_df['case_id'].unique()
        assert scenario_id in scenario_id_list
        return recording_df.loc[recording_df['case_id'] == scenario_id]

In [4]:
class ScenarioData:
    def __init__(self, scenario_df) -> None:
        self.scenario_df = scenario_df
        self.agent_num = self.get_num_of_agent()
        self.duration = self.get_total_time()
        self.vehicle_df_dict = self.get_vehicle_df_dict()
        self.inclusive_timestamps = self.get_inclusive_timestamps()
        

    def get_num_of_agent(self):
        return max(self.scenario_df['track_id'].unique())

    # get the starting and ending timestamps that includes all the agent in the scenario dataframe
    def get_inclusive_timestamps(self):
        min_timestamp = 0
        max_timestamp = self.duration
        for vid, vehicle_df in self.vehicle_df_dict.items():
            vehicle_min_timestamp = min(vehicle_df['timestamp_ms'])
            vehicle_max_timestamp = max(vehicle_df['timestamp_ms'])
            if vehicle_min_timestamp > min_timestamp:
                min_timestamp = vehicle_min_timestamp
            if vehicle_max_timestamp < max_timestamp:
                max_timestamp = vehicle_max_timestamp
        return [min_timestamp, max_timestamp]

    def get_total_time(self):
        return max(self.scenario_df['timestamp_ms'])

    def get_vehicle_df_dict(self):
        vehicle_df_dict = {}
        for i in range(1,self.agent_num+1):
            vehicle_df_dict[i] = self.scenario_df.loc[self.scenario_df['track_id'] == i]
        return vehicle_df_dict

    def construct_gs_file(self, saving_dir, start_sim_time):
        # const variable. Distance is measured in meters.
        origin = [0,0]
#         origin_icon_position = [0,0]
#         globalconfig_icon_position = [1,1]
        scenario_name = 'default scenario name'
        collision = True
        scenario_timestamp_df = self.scenario_df.loc[self.scenario_df['timestamp_ms'] == start_sim_time*1000]
        gsw = GSWriter()
        try:
            if start_sim_time < self.inclusive_timestamps[0] or start_sim_time > self.inclusive_timestamps[1]:
                raise Exception
        except Exception:
            warn('One or more vehicles are out of recording range under the starting simulation time.')
        
        try: 
            if start_sim_time > self.duration/1000.0:
                raise Exception
        except Exception:
            error('Starting simulation time is out of recording range.')
            return None
        
        timeout = self.duration/1000.0 - start_sim_time
            
        globalconfig_icon_latlon = gsw.wgs84ToUtm(1, 1)
        gsw.addGlobalConfig(globalconfig_icon_latlon[0], # global configuration icon position
                            globalconfig_icon_latlon[1], 
                            scenario_name,
                            self.get_map_directory(recording), # lanelet map directory
                            collision,
                            timeout)
        for i in range(scenario_timestamp_vehicle_num):
            agent_timestamp_df = scenario_timestamp_df.loc[scenario_timestamp_df["track_id"]==i+1]
            x = agent_timestamp_df['x']
            y = agent_timestamp_df['y']
            lat, lon = gsw.wgs84ToUtm(x, y)
            
            gsw.addVehicle(
                vehicle_name = 'v'+str(i),
                route_name = 'v'+str(i)+'_route', 
                behavior_type = 'SDV', 
                starting_yaw_deg = '17', 
                trajectory_lat = [0.00861226435, 0.00855672357], 
                trajectory_lon = [0.00907512695, 0.00989771443], 
                icon_lat = 0.00861980361, 
                icon_lon = 0.00904596863, 
                vehicle_id = 1, 
                behavior_tree_dir = 'drive.btree')

In [12]:
ID = InteractionDataset(dataset_config, train=False)
recording_df = ID.get_recording_df('DR_DEU_Merging_MT')
scenario_df = ID.get_scenario_df_of_recording(10, recording_df)
SD = ScenarioData(scenario_df)

In [29]:
SD.get_vehicle_df_dict()[1]

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width
1995,10.0,1,1,100,car,923.317,1005.331,-9.474,0.731,3.065,4.21,1.8
1999,10.0,1,2,200,car,922.370,1005.407,-9.513,0.762,3.062,4.21,1.8
2003,10.0,1,3,300,car,921.419,1005.487,-9.554,0.789,3.059,4.21,1.8
2007,10.0,1,4,400,car,920.464,1005.569,-9.596,0.814,3.057,4.21,1.8
2011,10.0,1,5,500,car,919.504,1005.653,-9.639,0.836,3.055,4.21,1.8
2015,10.0,1,6,600,car,918.541,1005.738,-9.682,0.856,3.053,4.21,1.8
2019,10.0,1,7,700,car,917.573,1005.826,-9.723,0.874,3.052,4.21,1.8
2023,10.0,1,8,800,car,916.601,1005.915,-9.762,0.890,3.051,4.21,1.8
2027,10.0,1,9,900,car,915.624,1006.006,-9.798,0.904,3.050,4.21,1.8
2031,10.0,1,10,1000,car,914.645,1006.097,-9.831,0.915,3.049,4.21,1.8


In [6]:
pd.read_csv('/mnt/Data/Research_Dataset/INTERACTION-Dataset-DR-multi-v1_2/val/DR_DEU_Merging_MT_val.csv')

,case_id,track_id,frame_id,timestamp_ms,agent_type,x,y,vx,vy,psi_rad,length,width
0,1.0,1,1,100,car,908.414,1006.631,-8.165,0.685,3.058,4.66,1.88
1,1.0,2,1,100,car,923.506,1005.483,-6.397,0.423,3.076,3.83,1.70
2,1.0,3,1,100,car,931.861,1004.774,-5.793,0.332,3.084,4.32,1.77
3,1.0,1,2,200,car,907.598,1006.699,-8.217,0.683,3.059,4.66,1.88
4,1.0,2,2,200,car,922.867,1005.527,-6.399,0.430,3.074,3.83,1.70
...,...,...,...,...,...,...,...,...,...,...,...,...
91605,343.0,3,39,3900,car,924.016,1005.576,-6.450,0.324,3.091,4.15,1.82
91606,343.0,4,39,3900,car,939.835,1004.726,-5.709,0.034,3.136,4.29,1.75
91607,343.0,2,40,4000,car,903.756,1007.267,-6.986,0.626,3.052,4.10,1.77
91608,343.0,3,40,4000,car,923.371,1005.611,-6.502,0.340,3.089,4.15,1.82


In [2]:
np.rad2deg(3.058)

175.21049375100574